# WNTR Fire Flow Tutorial
The following tutorial covers how to run simple fire flow analysis using WNTR.

## Imports
Import WNTR and additional Python packages that are needed for the tutorial.
- Numpy is required to define comparison operators (i.e., np.greater) in queries
- Pandas is used for data manipulation and analysis
- Matplotlib is used to create graphics

In [ ]:
import wntr
import matplotlib
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

## Water network model
If adapting code for a different EPANET input (INP) file, the correct file path and desired simulation parameters will need to be updated.

In [ ]:
# Identify file path to inp file
inp_file = "networks/Net3.inp"

# Create water network model 
wn = wntr.network.WaterNetworkModel(inp_file)

# Calculate population per junction
population = wntr.metrics.population(wn)

## Fire flow parameters
The parameters `minimum_pressure` and `required_pressure` are used for pressure dependent demand (PDD) simulations. Nodes with pressures below minimum pressure will not receive any water, and node pressures need to be at least the required pressure to receive all of the requested demand.

Assuming that hydrants are at every junction in the network model, hydrants of interest are identified as the nodes connnected to pipes with diameters of interest. For this reason, a range of pipe diameters to include in the analysis needs to be identified.

In [ ]:
# Define simulation parameters 
start_time = 2*3600 # 2 hours
fire_duration = 4*3600 # 4 hours
total_duration = start_time + fire_duration

fire_demand = 0.5047 # 8000 GPM

minimum_pressure = 3.52 # 5 psi
required_pressure = 14.06 # 20 psi

# Identify the range of pipe diameters
min_pipe_diam = 0.1524 # 6 inch
max_pipe_diam = 0.2032 # 8 inch

## Baseline simulation
The baseline simulation can be used to identify non-zero (NZD) junctions that fall below minimum pressure during normal operating conditions. This step helps determine which junctions that experience low pressures during the disaster simulation are a direct result of the disaster and not normal operating conditions. 

In [ ]:
# Calculate average expected demand (AED) and identify junctions with non-zero AED
AED = wntr.metrics.average_expected_demand(wn)
nzd_junct = AED[AED > 0].index

# Set hydraulic parameters
wn.options.hydraulic.demand_model = 'PDD'    
wn.options.time.duration = total_duration
wn.options.hydraulic.minimum_pressure = minimum_pressure
wn.options.hydraulic.required_pressure = required_pressure 

# Simulate the hydraulics
sim = wntr.sim.WNTRSimulator(wn)
results = sim.run_sim()

# Save junction pressure results and identify junctions that fall below minimum pressure
pressure = results.node['pressure'].loc[start_time::, nzd_junct]
normal_pressure_below_pmin = pressure.columns[(pressure < minimum_pressure).any()]

## Fire flow simulations 
A try/except/finally approach is taken to ensure the script can finish running and still catch any convergence issues that might have occurred due to increased fire flow demand. A user can revisit nodes with failed simulations individually to determine the cause of failure, if desired.

In [ ]:
# Query pipes with diameters within diameter bounds set earlier to include in analysis
pipe_diameter = wn.query_link_attribute('diameter')
pipes_of_interest = pipe_diameter[(pipe_diameter <= max_pipe_diam) & (pipe_diameter >= min_pipe_diam)]

In [ ]:
# Identify junctions connected to pipes of interest
junct_of_interest = set()
for pipe_name in pipes_of_interest.index:
    pipe = wn.get_link(pipe_name)
    if pipe.start_node_name in wn.junction_name_list:
        junct_of_interest.add(pipe.start_node_name)
    if pipe.end_node_name in wn.junction_name_list:
        junct_of_interest.add(pipe.end_node_name)

In [ ]:
# Create dictionary to save results
analysis_results = {}

# Simulate fire flow demand for each hydrant location
for junct in junct_of_interest:
    wn = wntr.network.WaterNetworkModel(inp_file)
    wn.options.hydraulic.demand_model = 'PDD'    
    wn.options.time.duration = total_duration
    wn.options.hydraulic.minimum_pressure = minimum_pressure
    wn.options.hydraulic.required_pressure = required_pressure

    # Create fire flow pattern
    fire_flow_pattern = wntr.network.elements.Pattern.binary_pattern(
        'fire_flow',
        start_time=start_time,
        end_time=total_duration,
        step_size=wn.options.time.pattern_timestep,
        duration=wn.options.time.duration
        )
    wn.add_pattern('fire_flow', fire_flow_pattern)

    # Apply fire flow pattern to hydrant location
    fire_junct = wn.get_node(junct)
    fire_junct.demand_timeseries_list.append((fire_demand, fire_flow_pattern, 'Fire flow'))

    try:
        # Simulate hydraulics
        sim = wntr.sim.WNTRSimulator(wn) 
        sim_results = sim.run_sim()
 
        # Identify impacted junctions using pressure results
        sim_pressure = sim_results.node['pressure'].loc[start_time::, nzd_junct]
        sim_pressure_below_pmin = sim_pressure.columns[(sim_pressure < minimum_pressure).any()]
        impacted_junctions = set(sim_pressure_below_pmin) - set(normal_pressure_below_pmin)
        impacted_junctions = list(impacted_junctions)
        
    except Exception as e:
        # Identify failed simulations and the reason
        impacted_junctions = None
        print(junct, ' Failed:', e)

    finally:
        # Save simulation results
        analysis_results[junct] = impacted_junctions

## Results

In [ ]:
# Calculate and save junction and population impact results to dictionary
num_junctions_impacted = {}
num_people_impacted = {}
for pipe_name, impacted_junctions in analysis_results.items():
    if impacted_junctions is not None:
        num_junctions_impacted[pipe_name] = len(impacted_junctions)
        num_people_impacted[pipe_name] = population[impacted_junctions].sum()

In [ ]:
# Set colormap for network maps
cmap=matplotlib.colormaps['viridis']

# Plot junctions impacted due to increased fire flow
# The parameter `node_range` can be adjusted to better suit the simulation results of the network used in the analysis
wntr.graphics.plot_network(wn, node_attribute=num_junctions_impacted, node_size=20, link_width=0, 
                           node_range=[0,5], node_cmap = cmap, node_colorbar_label='Junctions Impacted', 
                           title='Number of junctions impacted by each fire flow demand')

# Plot population impacted due to increased fire flow
wntr.graphics.plot_network(wn, node_attribute=num_people_impacted, node_size=20, link_width=0, 
                           node_range=[0,2500], node_cmap = cmap, node_colorbar_label='Population Impacted',
                           title='Number of people impacted by each fire flow demand')                    

## Save results to CSV files

In [ ]:
# Save the junction impacted results to CSV
# Check to verify the file was created in the directory
num_junctions_impacted = pd.Series(num_junctions_impacted)
num_junctions_impacted.to_csv('fire_flow_junctions_impacted.csv')

# Save the population impacted results to CSV
num_people_impacted = pd.Series(num_people_impacted)
num_people_impacted.to_csv('fire_flow_people_impacted.csv')